<a href="https://colab.research.google.com/github/41371109h/114-1/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q google-generativeai

In [41]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [42]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('wt')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [43]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('wt')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [44]:
import pandas as pd
import gspread
import datetime
import gradio as gr
from google.colab import auth
from google.auth import default

# --- Google Sheet 認證與連接 ---
# 【重要】請確保此單元格完整執行，以完成 Google 授權
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Google 認證成功！")

# 連接 Google Sheet
# 【請替換成你目前使用的 Google Sheets 網址】
sheet_url = 'https://docs.google.com/spreadsheets/d/1Z5JyALae6Y8ioWc4--aTTB8pjXI5f6eBif33KDIERWM/edit?gid=0#gid=0'
try:
    gsheets = gc.open_by_url(sheet_url)
    worksheet = gsheets.worksheet('工作表1') # 請確認你的工作表名稱
    print(f"✅ Google Sheets 檔案已成功連接：{gsheets.title}")
except Exception as e:
    print(f"❌ 連接 Google Sheets 時發生錯誤：{e}")
    # 這裡拋出錯誤，讓程式停止，避免後續操作失敗
    raise

# --- 函數區：用於 Gradio 介面呼叫 ---

def get_grades_df():
    """從 Google Sheet 讀取所有成績並轉換為 DataFrame"""
    try:
        sheets_data = worksheet.get_all_values()
        # 假設第一列是標題
        df = pd.DataFrame(sheets_data[1:], columns=sheets_data[0])
        return df
    except Exception as e:
        # 如果讀取失敗，回傳空的 DataFrame
        return pd.DataFrame()

def record_and_update_grade(student_name, subject, grade):
    """
    將單一筆成績紀錄寫入 Google Sheet，並回傳訊息和更新後的 DataFrame。
    """
    # 1. 檢查輸入是否為空
    if not student_name or not subject or not grade:
        return "⚠️ 錯誤：姓名、科目和成績都不能為空。", get_grades_df()

    # 2. 驗證成績：確保是數字
    try:
        float(grade)
    except ValueError:
        return "⚠️ 錯誤：成績必須是數字。請重新輸入！", get_grades_df()

    # 3. 準備寫入資料
    date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    new_row = [date_str, student_name, subject, grade]

    # 4. 寫入 Google Sheet
    try:
        worksheet.append_row(new_row)
        # 寫入成功後，重新讀取資料並回傳 (實現即時更新)
        return "✅ 成功新增一筆成績紀錄！", get_grades_df()
    except Exception as e:
        return f"❌ 寫入 Google Sheet 時發生錯誤：{e}", get_grades_df()

# --- Gradio 介面區 ---
with gr.Blocks(title="成績即時紀錄系統") as demo:
    gr.Markdown("# 🏆 成績即時紀錄系統 (Google Sheets 連動)")
    gr.Markdown("在下方輸入成績，點擊『記錄成績』，資料會**即時**寫入 Google Sheets 並更新表格。")

    # 修正後的代碼：移除 'live=True'
    # 'value=get_grades_df' 會在介面載入時執行，但後續更新由 submit_button.click 負責
    grades_table = gr.DataFrame(value=get_grades_df, label="現有成績資料 (即時更新)")

    gr.Markdown("---")

    # 介面輸入框
    with gr.Row():
        student_name_input = gr.Textbox(label="學生姓名", placeholder="例如：李小華")
        subject_input = gr.Textbox(label="科目", placeholder="例如：英文")
        grade_input = gr.Textbox(label="成績", placeholder="例如：88")

    # 按鈕和回饋訊息
    with gr.Row():
        submit_button = gr.Button("記錄成績")
        output_message = gr.Textbox(label="操作訊息", interactive=False, value="")

    # 設定按鈕點擊事件：點擊後，更新 output_message 和 grades_table
    submit_button.click(
        fn=record_and_update_grade,
        inputs=[student_name_input, subject_input, grade_input],
        outputs=[output_message, grades_table]
    )

if __name__ == "__main__":
    demo.launch(share=True)

✅ Google 認證成功！
✅ Google Sheets 檔案已成功連接：HW2_成績一本通
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d59d3a7d18b059303a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
